## Tutorial: Flux balance analysis

In this tutorial we will use `COBREXA`'s `flux_balance_analysis` function to perform flux balance analysis on a toy model of *E. coli*.

Let's first load the model.

In [1]:
# download file if it is not already present
!isfile("e_coli_core.xml") && download("http://bigg.ucsd.edu/static/models/e_coli_core.xml", "e_coli_core.xml")

using COBREXA

model = load_model("e_coli_core.xml")

Metabolic model of type SBMLModel

⠀⠈⢀⠀⡀⠀⠀⠀⠀⡠⠂⠀⠀⠀⠀⠈⠀⠄⠀⠀⠀⠀⠀⠀⠀⠀⠀⠠⠀⠀⠀⠀⢀⠐⡀⠀⠀⠀⠀⠄
⠀⠐⠀⠀⠀⠀⠀⠀⡠⠂⠀⠀⠀⠀⢰⠱⣀⠀⡄⢐⠀⠀⢀⠀⠀⠀⡂⠄⠔⠁⠰⠀⠠⠀⣆⠀⠄⢠⢀⠄
⠀⠀⠀⠀⠀⠀⠀⠀⠀⠁⠠⠀⠀⠐⠀⠀⠀⠀⠀⠀⢀⠀⠀⠐⠀⠂⠀⠀⠀⠄⠀⠐⠀⢁⠄⠀⠀⠀⠀⠀
⠀⢀⠀⠐⡈⠀⡀⠀⠂⠀⣀⠀⠑⡈⢀⠀⠀⠀⠀⠀⡀⡠⠀⡀⠰⠁⠈⠂⠁⠀⠠⠀⠀⠂⡂⠀⠂⠂⠀⠀
⠠⠀⠐⠀⠂⠀⠀⢀⠀⠀⠀⠀⠊⠀⡐⠊⠐⠀⠀⠀⠀⠀⠐⠀⠂⠀⠀⠐⠀⠀⠀⠀⠀⠁⠃⠠⠀⠁⠐⠀
⠀⠠⠀⡀⠄⠀⠀⠂⠀⠀⠀⠠⠀⠠⠀⠀⠄⠀⠨⠀⠀⠀⠐⠀⠀⠄⢀⠀⠀⠀⠈⠀⠀⠀⠁⠄⠀⠀⠀⠀
⠀⢐⠐⠀⠄⠀⡂⠀⢐⠀⠀⠀⠀⠂⢀⢀⠐⠂⡀⠈⠀⠀⠀⠂⠀⠈⠀⡀⡐⠀⢄⠀⢀⠀⡆⠀⡀⣀⡀⡐
⠀⠈⠀⠀⠀⠀⠀⠐⢂⠀⢀⠀⠈⠀⠀⠀⠀⠀⠠⠀⠀⠠⠀⠀⠀⠈⠂⠀⠀⠀⠄⠐⠐⠀⠁⠀⠀⠑⠁⠀
⠂⠠⠀⠀⠀⠀⠀⠀⠀⢀⠀⠀⠠⠈⠀⠀⠀⠀⠀⠁⠀⠀⠠⠐⠀⠁⠈⠀⠁⢀⠀⠀⠀⠀⠀⠀⠀⠀⠌⠀
⠀⠀⠂⢨⠀⡀⠀⠐⠁⠐⠀⠐⠊⠀⠀⠀⠀⠀⠀⠀⠀⠀⠢⠒⠈⠐⠐⠁⠂⠀⠀⠀⠄⠓⠕⠂⠃⠁⠀⠐
⠠⠀⠨⠀⠁⠤⠄⠀⠁⡄⠀⠂⠠⠄⢈⠌⠠⠄⠀⢀⠀⠀⠀⠄⠨⠀⡤⠀⢀⠀⢀⠠⠀⠁⡔⠨⠀⠈⠄⠀
⠀⢀⢀⣀⠀⡠⡒⢀⢀⣀⠀⢀⣀⡀⢀⠀⢀⠀⡀⠀⡀⠀⠈⣀⠀⢀⣀⠀⡀⠀⢀⠁⢀⣀⣀⡀⡠⡀⡀⣀
⠀⠄⠀⠀⠀⠀⠀⠀⠀⠂⠁⠀⠀⠀⠀⠀⠀⣠⠀⠀⡀⠀⠀⠀⠀⠀⠀⠀⠈⠀⠀⠀⠀⠀⠀⠐⠀⠀⠀⠀
⢀⠂⠀⠀⠂⠀⠈⠀⠐⠀⠀⠀⠁⠀⠀⠀⡀⠔⠑⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠢⠀⠀⡀⠂⠈⠀⠀⠀⠄
⠀⠐⠀⠀⡂⠀⠂⠀⠀⠀⠒⠐⠄⠂⠐⠀⠘⡀⠀⠠⡂⠃⠀⠂⠄⠂⠀⠀⠀⠀⡀⠀⡀⠀⡂⠂⠀⠀⢀⠀
Number of reactions: 95
Number of metabolites: 72


### Basic FBA

To actually perform any optimization based analysis we need to load a solver. Any [`JuMP` supported solvers](https://jump.dev/JuMP.jl/stable/installation/#Supported-solvers) will work, but here we will use [`Tulip.jl`](https://github.com/ds4dm/Tulip.jl) for convenience. If you do not have `Tulip.jl` installed, you can do so with `] add Tulip`.

In [2]:
using Tulip

optimizer = Tulip.Optimizer # in general the format is <OptimizerName>.Optimizer

Tulip.Optimizer

Most analysis functions come in two flavours that determine how the output will be returned. The suffix of the function determines this behaviour, e.g. `flux_balance_analysis_vec` returns a vector of fluxes in the order of the reactions returned by `reactions` (recall that this is one of the generic interface accessors), and `flux_balance_analysis_dict` that returns a dictionary mapping reaction ids to fluxes. 

In all cases `flux_balance_analysis` functions takes two required inputs: the `model` and the `optimizer`.

In [3]:
vec_soln = flux_balance_analysis_vec(model, optimizer)

95-element Vector{Float64}:
  -0.0
   6.007249566587288
   7.477381919012616
  -5.064375360924965
   0.2234617471308555
  -3.2148950303940156
   2.5043094327812505
  21.799492758453965
   4.959985078558465
   1.4969838027636577
   5.883903449309813e-8
   2.3921385258661974e-9
   4.959985078558436
   ⋮
   2.5510067318203676e-8
 -22.80983339309117
   6.007249566587296
   3.374152410464434e-7
  29.17582720267984
   9.04807316633683e-9
   4.819851209389017e-8
   9.952893763995706e-9
 -21.799492758453965
  -0.0
  -1.4330520742992535e-9
   3.2148950303940156

In [4]:
dict_soln = flux_balance_analysis_dict(model, optimizer)

Dict{String, Float64} with 95 entries:
  "R_EX_fum_e"    => -0.0
  "R_ACONTb"      => 6.00725
  "R_TPI"         => 7.47738
  "R_SUCOAS"      => -5.06438
  "R_GLNS"        => 0.223462
  "R_EX_pi_e"     => -3.2149
  "R_PPC"         => 2.50431
  "R_O2t"         => 21.7995
  "R_G6PDH2r"     => 4.95999
  "R_TALA"        => 1.49698
  "R_PPCK"        => 5.8839e-8
  "R_EX_lac__D_e" => 2.39214e-9
  "R_PGL"         => 4.95999
  "R_H2Ot"        => -29.1758
  "R_GLNabc"      => -0.0
  "R_EX_co2_e"    => 22.8098
  "R_EX_gln__L_e" => -0.0
  "R_EX_nh4_e"    => -4.76532
  "R_MALt2_2"     => -0.0
  "R_ME2"         => 1.49736e-7
  "R_GAPD"        => 16.0235
  "R_EX_akg_e"    => 3.85074e-9
  "R_CS"          => 6.00725
  "R_ETOHt2r"     => -1.43306e-9
  "R_ACKr"        => -3.41463e-9
  ⋮               => ⋮

### Problem modification

Often it is desirable to modify the problem slightly before performing analysis. Problem modifications include things like changing the objective sense, the solver, the solver attributes, the flux constraints, and the optimization objective. For completeness we demonstrate all of the mentioned problem modifications below. The constraints are added to the `flux_balance_analysis_vec` (or `flux_balance_analysis_dict`) function call as optional keyword arguments.

Note that these modifications are temporary and do not change the underlying `model`. Also note that we are making use of another optimization package, `GLPK.jl` to illustrate the `change_solver` modification function. While the `change_solver` option might seem unnecessary here, it is useful when solving parsimonious flux balance analysis type problems (pFBA), as demonstrated in another tutorial.

In [5]:
using GLPK # add this solver using ] add GLPK (if you don't already have it)

dict_sol = flux_balance_analysis_dict(
    model,
    GLPK.Optimizer; # will change to Tulip below
    modifications = [ # modifications are evaluated in order
        change_objective("R_BIOMASS_Ecoli_core_w_GAM"),
        change_constraint("R_EX_glc__D_e", -12, -12),
        change_constraint("R_EX_o2_e", 0, 0),
        change_solver(Tulip.Optimizer), # swap back to using Tulip
        change_solver_attribute("IPM_IterationsLimit", 110), # this is a Tulip specific attribute, other solvers have other attributes
        change_sense(COBREXA.MOI.MAX_SENSE), # COBREXA imports JuMP internally, other valid options here also include: COBREXA.MOI.MIN_SENSE
        ],
)

Dict{String, Float64} with 95 entries:
  "R_EX_fum_e"    => -0.0
  "R_ACONTb"      => 0.294088
  "R_TPI"         => 11.7289
  "R_SUCOAS"      => -5.31277e-11
  "R_GLNS"        => 0.069699
  "R_EX_pi_e"     => -1.00274
  "R_PPC"         => 0.781108
  "R_O2t"         => -2.65313e-17
  "R_G6PDH2r"     => 4.23154e-9
  "R_TALA"        => -0.0487648
  "R_PPCK"        => 5.66084e-11
  "R_EX_lac__D_e" => 1.86325e-10
  "R_PGL"         => 4.23153e-9
  "R_H2Ot"        => 8.2857
  "R_GLNabc"      => -0.0
  "R_EX_co2_e"    => -0.487021
  "R_EX_gln__L_e" => -0.0
  "R_EX_nh4_e"    => -1.48633
  "R_MALt2_2"     => -0.0
  "R_ME2"         => 1.40825e-10
  "R_GAPD"        => 23.2754
  "R_EX_akg_e"    => 6.82418e-11
  "R_CS"          => 0.294088
  "R_ETOHt2r"     => -9.78427
  "R_ACKr"        => -10.0729
  ⋮               => ⋮

### Using syntactic sugar 
Some functions are also available in macro format. This is purely for cosmetic reasons and adds no functionality beyond that. The previous example can be rewritten as shown below:

In [6]:
dict_sol = @flux_balance_analysis_dict model GLPK.Optimizer begin # notice that no commas are necessary here
        change_objective("R_BIOMASS_Ecoli_core_w_GAM")
        change_constraint("R_EX_glc__D_e", -12, -12)
        change_constraint("R_EX_o2_e", 0, 0)
        change_solver(Tulip.Optimizer)
        change_solver_attribute("IPM_IterationsLimit", 110)
        change_sense(COBREXA.MOI.MAX_SENSE)
end

Dict{String, Float64} with 95 entries:
  "R_EX_fum_e"    => -0.0
  "R_ACONTb"      => 0.294088
  "R_TPI"         => 11.7289
  "R_SUCOAS"      => -5.31277e-11
  "R_GLNS"        => 0.069699
  "R_EX_pi_e"     => -1.00274
  "R_PPC"         => 0.781108
  "R_O2t"         => -2.65313e-17
  "R_G6PDH2r"     => 4.23154e-9
  "R_TALA"        => -0.0487648
  "R_PPCK"        => 5.66084e-11
  "R_EX_lac__D_e" => 1.86325e-10
  "R_PGL"         => 4.23153e-9
  "R_H2Ot"        => 8.2857
  "R_GLNabc"      => -0.0
  "R_EX_co2_e"    => -0.487021
  "R_EX_gln__L_e" => -0.0
  "R_EX_nh4_e"    => -1.48633
  "R_MALt2_2"     => -0.0
  "R_ME2"         => 1.40825e-10
  "R_GAPD"        => 23.2754
  "R_EX_akg_e"    => 6.82418e-11
  "R_CS"          => 0.294088
  "R_ETOHt2r"     => -9.78427
  "R_ACKr"        => -10.0729
  ⋮               => ⋮

### Advanced usage - typically unnecessary
Sometimes it is useful to have access to the solved `JuMP` optimization model instead of mapping the answers to vectors or dictionaries. `COBREXA` allows you to do this using the unmodified `flux_balance_analysis` function like so:

In [7]:
opt_model = flux_balance_analysis(model, optimizer) # notice that the suffix has been removed

A JuMP Model
Maximization problem with:
Variables: 95
Objective function type: JuMP.AffExpr
`JuMP.AffExpr`-in-`MathOptInterface.EqualTo{Float64}`: 73 constraints
`JuMP.AffExpr`-in-`MathOptInterface.LessThan{Float64}`: 192 constraints
Model mode: AUTOMATIC
CachingOptimizer state: ATTACHED_OPTIMIZER
Solver name: Tulip
Names registered in the model: lbs, mb, ubs, x

Here `opt_model` is a `JuMP` model structure. You can access the `JuMP` variables using `opt_model[:x]`, the lower bound constraints using `opt_model[:lbs]`, the upper bound constraints using `opt_model[:ubs]`, and the mass balance constraints using `opt_model[:mb]`.

This type of usage is generally not required, but situations could arise where it is useful. 

We also provide a helper function `set_bound` that can be used to modify the upper and lower bound constraints of a `JuMP` model returned by `COBREXA` functions directly, e.g. `opt_model` in this example. This function is useful because the `JuMP` function `set_normalized_rhs` can be slightly confusing to use since it normalizes constraints, see `JuMP`'s [documentation](https://jump.dev/JuMP.jl/v0.21.1/constraints/#Constraint-modifications-1) for more details. Regardless, we recommend that you use `COBREXA`'s `set_bound` function since it is much more transparent.

In [8]:
?set_bound

search: set_bound



```
set_bound(index, optimization_model;
    ub=_constants.default_reaction_rate,
    lb=-_constants.default_reaction_rate)
```

Helper function to set the bounds of variables. The JuMP `set_normalized_rhs` function is a little confusing,  so this function simplifies setting constraints. In short, JuMP uses a normalized right hand side representation of constraints,  which means that lower bounds have their sign flipped. This function does this for you, so you don't have to remember to do this whenever you change the constraints. 

Just supply the constraint `index` and the JuMP model (`opt_model`) that  will be solved, and the variable's bounds will be set to `ub` and `lb`.


In [9]:
set_bound(first(indexin(["R_EX_glc__D_e"], reactions(model))), opt_model; lb = -4, ub=-4) # change glucose flux constraint

COBREXA.JuMP.optimize!(opt_model) # optimize model using new bound
COBREXA.JuMP.value(opt_model[:x][first(indexin(["R_BIOMASS_Ecoli_core_w_GAM"], reactions(model)))]) # biomass flux

0.3239330285651231